In [152]:
import numpy as np
from random import randint
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Embedding, Dropout
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/kushal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('gutenberg')
from nltk.corpus import gutenberg
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/kushal/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [4]:
macbeth_text = nltk.corpus.gutenberg.raw('shakespeare-macbeth.txt')

In [6]:
print(macbeth_text[:500])

[The Tragedie of Macbeth by William Shakespeare 1603]


Actus Primus. Scoena Prima.

Thunder and Lightning. Enter three Witches.

  1. When shall we three meet againe?
In Thunder, Lightning, or in Raine?
  2. When the Hurley-burley's done,
When the Battaile's lost, and wonne

   3. That will be ere the set of Sunne

   1. Where the place?
  2. Vpon the Heath

   3. There to meet with Macbeth

   1. I come, Gray-Malkin

   All. Padock calls anon: faire is foule, and foule is faire,
Houer through 


In [73]:
def clean_text(text):
    sentence = re.sub('[^a-zA-Z]', ' ', text)
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    sentence = re.sub('\s+', ' ', sentence)
    return sentence.lower()

In [74]:
temp = clean_text(macbeth_text)
macbeth_text_processed = temp[1:]
print(macbeth_text_processed[:500])

the tragedie of macbeth by william shakespeare actus primus scoena prima thunder and lightning enter three witches when shall we three meet againe in thunder lightning or in raine when the hurley burley done when the battaile lost and wonne that will be ere the set of sunne where the place vpon the heath there to meet with macbeth come gray malkin all padock calls anon faire is foule and foule is faire houer through the fogge and filthie ayre exeunt scena secunda alarum within enter king malcome


In [75]:
macbeth_word_tokenized = word_tokenize(macbeth_text_processed)
print("The text has %i words and %i unique words"%(len(macbeth_word_tokenized), len(set(macbeth_word_tokenized))))

The text has 17250 words and 3436 unique words


In [100]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = len(set(macbeth_word_tokenized)))
tokenizer.fit_on_texts(macbeth_word_tokenized)

vocab_size = len(tokenizer.word_index)
word_2_index = tokenizer.word_index

In [103]:
print(macbeth_word_tokenized[500])
print(word_2_index[macbeth_word_tokenized[500]])

comparisons
1456


In [137]:
input_sequence = [] 
output_sequence = []

sequence_length = 100

for i in range(len(macbeth_word_tokenized) - sequence_length):
    temp_input = macbeth_word_tokenized[i:i+sequence_length]
    temp_output = macbeth_word_tokenized[i+sequence_length]
    input_sequence.append([word_2_index[j] for j in temp_input])
    output_sequence.append([word_2_index[temp_output]])

In [145]:
import tensorflow as tf
X = np.reshape(input_sequence, (len(input_sequence), sequence_length, 1))
X = X / len(set(macbeth_word_tokenized))
y = tf.keras.utils.to_categorical(output_sequence)
print(X.shape)
print(y.shape)


(17150, 100, 1)
(17150, 3437)


In [167]:
model = Sequential()
model.add(LSTM(800, input_shape = (X.shape[1], X.shape[2]), return_sequences = True))
model.add(LSTM(800, return_sequences = True))
model.add(LSTM(800))
model.add(Dense(y.shape[1], activation = 'softmax'))
model.summary()
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')


Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_29 (LSTM)               (None, 100, 800)          2566400   
_________________________________________________________________
lstm_30 (LSTM)               (None, 100, 800)          5123200   
_________________________________________________________________
lstm_31 (LSTM)               (None, 800)               5123200   
_________________________________________________________________
dense_7 (Dense)              (None, 3437)              2753037   
Total params: 15,565,837
Trainable params: 15,565,837
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X,y, batch_size = 64, epochs = 10)